In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from typing import Optional, Iterable
import gradio as gr
import json

load_dotenv(override=True)
# openai_api_key = os.getenv("OPENAI_API_KEY")

media_dic = {
    "title": "Understanding the Ovaries: Functions and Importance",
    "video": "https://www.youtube.com/watch?v=XEjekWehWas",
    "pdf": "ana.pdf"
}


In [2]:
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()

summarize_system_prompt = """You are an academic summarizer. 
Read the teacher’s transcribed text carefully and produce a clear, concise, 
and well-structured summary. Focus on the main ideas, learning objectives, 
and key explanations. Remove filler words, repetitions, and irrelevant dialogue.
The summary should sound educational, easy to understand, 
and suitable for student notes, if you dont know the 
answer please say so."""

user_explain_system_prompt = """You are a helpful educational assistant, 
that explains any concept clearly and simply based on the user’s specific 
challenge, confusion, or question. Always ensure the explanation is easy to 
understand, uses examples, and connects directly to the user’s difficulty.
Identify what part of the concept the user finds difficult or confusing,
Explain it step-by-step using clear, everyday language 
Give practical examples, analogies, or comparisons to make it relatable"""

list_points_system_prompt = """You are an expert at converting text into
a list of key points. Read the provided text carefully and extract the main
ideas, concepts, or steps. Present these in a clear, concise list format.
Ensure each point is distinct and captures the essence of the original text.
Avoid unnecessary details or filler information. The list should be easy to
read and understand, suitable for quick reference or study purposes.
Respond only with the list of key points.
"""

flash_card_point_system_prompt = """ """

def summarize_transcript(transcript):
    messages = [
        {"role": "system", "content": summarize_system_prompt},
        {"role": "user", "content": transcript}
    ]
    
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0.2,
        max_tokens=500,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    
    summary = response.choices[0].message.content
    return summary


def user_explain_concept(concept, user_question):
    messages = [
        {"role": "system", "content": user_explain_system_prompt},
        {"role": "user", "content": f"Concept: {concept}\nUser Question: {user_question}"}
    ]
    
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0.3,
        max_tokens=500,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    
    explanation = response.choices[0].message.content
    return explanation

def list_key_points(text):
    messages = [
        {"role": "system", "content": list_points_system_prompt},
        {"role": "user", "content": text}
    ]
    
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0.2,
        # max_tokens=300,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    
    return response.choices[0].message.content



   




OpenAI API Key exists and begins sk-proj-


In [4]:
def _try_import(name: str):
    try:
        return __import__(name)
    except Exception as e:
        raise ImportError(f"Missing dependency '{name}': {e}")

PyPDF2 = _try_import("PyPDF2")

def transcribe_pdf_to_text(
    pdf_path: str,
    ocr: bool = False,
    pages: Optional[Iterable[int]] = None,
    dpi: int = 300,
    write_to: Optional[str] = None,
) -> str:
    """
    Extract text from a PDF file. Uses direct PDF text extraction first.
    If ocr=True or no text found, falls back to OCR (pytesseract + pdf2image).

    Args:
      pdf_path: path to the PDF file
      ocr: force OCR fallback or allow OCR when no text found
      pages: iterable of 0-based page indices to process (default: all)
      dpi: resolution for OCR page images
      write_to: optional path to write the resulting text

    Returns:
      The extracted text as a single string.
    """
    if not os.path.isfile(pdf_path):
        raise FileNotFoundError(pdf_path)

    reader = PyPDF2.PdfReader(pdf_path)
    if getattr(reader, "is_encrypted", False):
        try:
            reader.decrypt("")  # try empty password
        except Exception:
            raise RuntimeError("PDF is encrypted and cannot be decrypted automatically.")

    # Normalize pages selection
    total = len(reader.pages)
    if pages is None:
        page_indices = range(total)
    else:
        page_indices = [p for p in pages if 0 <= p < total]

    # 1) Try direct text extraction
    text_parts = []
    for i in page_indices:
        try:
            page = reader.pages[i]
            txt = page.extract_text() or ""
        except Exception:
            txt = ""
        text_parts.append(txt)

    full_text = "\n\n".join(part for part in text_parts if part and part.strip())

    # 2) If no text found and OCR allowed, do OCR
    if (not full_text.strip()) and ocr:
        pdf2image = _try_import("pdf2image")
        pytesseract = _try_import("pytesseract")
        from PIL import Image  # pillow

        # convert selected pages to images
        # pdf2image.convert_from_path accepts first_page/last_page (1-based)
        # We'll convert the minimal page range if pages is continuous, otherwise convert all and index.
        images = pdf2image.convert_from_path(pdf_path, dpi=dpi)
        ocr_texts = []
        for i in page_indices:
            img = images[i]
            ocr_texts.append(pytesseract.image_to_string(img))
        full_text = "\n\n".join(ocr_texts)

    if write_to:
        with open(write_to, "w", encoding="utf-8") as f:
            f.write(full_text)

    return full_text

pdf_text = transcribe_pdf_to_text(media_dic.get("pdf",""))

In [5]:
list_key_points(pdf_text) 
# user_explain_concept(pdf_text, "Dont understand everything")

'- Human anatomy is the scientific study of the structure of the human body and relationships among its parts.\n- It is one of the oldest sciences, originating in ancient Egypt and Greece.\n- Anatomy forms the foundation for medicine, physiology, and biological sciences related to human life.\n- Studying anatomy helps understand body system interactions and effects of diseases or injuries.\n\nLevels of Structural Organization:\n- Chemical level: atoms and molecules form the building blocks of life.\n- Cellular level: cells are the smallest units of life capable of independent function.\n- Tissue level: groups of similar cells with common functions (e.g., muscle tissue, nervous tissue).\n- Organ level: tissues combine to form organs (e.g., heart, lungs, liver) with specific tasks.\n- Organ system level: multiple organs work together in systems.\n- Organism level: all organ systems together form the human body.\n\nMajor Organ Systems:\n1. Integumentary System: skin, hair, nails, glands; 

In [ ]:
# transcript =""". 
# Sure — here’s a **50-line educational transcript** explaining the **human female sexual organs** clearly and respectfully, as it might sound if spoken by a teacher during a health or biology class:

# ---

# ### 🎓 **Transcript: The Human Female Sexual Organ**

# 1. Good morning, everyone.
# 2. Today, we’ll be learning about the **human female sexual organs**.
# 3. This topic is part of our reproductive health education.
# 4. The female reproductive system has both **external** and **internal** organs.
# 5. Each organ has a special role in reproduction and overall health.
# 6. Let’s start with the **external organs**, also known as the **vulva**.
# 7. The vulva is the part you can see from the outside of the body.
# 8. It includes several structures working together for protection and sensation.
# 9. The **mons pubis** is the soft, fatty area above the pubic bone.
# 10. It becomes covered with hair after puberty.
# 11. Below it are the **labia majora**, or “outer lips.”
# 12. These folds of skin protect the inner parts of the vulva.
# 13. Inside them are the **labia minora**, or “inner lips.”
# 14. They are more delicate and vary in size and color between individuals.
# 15. In the upper part of the vulva, you’ll find the **clitoris**.
# 16. The clitoris is a small, sensitive organ important for sexual pleasure.
# 17. It contains many nerve endings — more than any other part of the body.
# 18. The **urethral opening** lies just below the clitoris.
# 19. This is where urine passes out of the body.
# 20. Beneath it is the **vaginal opening**, also called the **introitus**.
# 21. The vaginal opening leads into the **vagina**, an internal muscular canal.
# 22. The vagina connects the external genitals to the uterus inside the body.
# 23. It also serves as the passage for menstrual flow and childbirth.
# 24. Now let’s move on to the **internal organs**.
# 25. The first internal structure is the **vagina**, as we just mentioned.
# 26. It’s followed by the **cervix**, which is the lower part of the uterus.
# 27. The cervix opens slightly to allow menstrual blood to flow out.
# 28. During childbirth, it expands to let the baby pass through.
# 29. Above the cervix is the **uterus**, also known as the womb.
# 30. The uterus is a hollow, pear-shaped organ made of strong muscles.
# 31. This is where a fertilized egg can implant and grow into a baby.
# 32. The lining of the uterus thickens each month in preparation for pregnancy.
# 33. If pregnancy doesn’t occur, this lining sheds during menstruation.
# 34. On either side of the uterus are two **fallopian tubes**.
# 35. The fallopian tubes transport eggs from the ovaries to the uterus.
# 36. Fertilization — when the sperm meets the egg — usually happens in these tubes.
# 37. At the end of each tube are tiny, finger-like projections called **fimbriae**.
# 38. The fimbriae help guide the egg into the tube after ovulation.
# 39. Finally, we have the **ovaries** — one on each side of the uterus.
# 40. The ovaries produce **eggs** (or ova) and important hormones like estrogen and progesterone.
# 41. These hormones control the menstrual cycle and female body development.
# 42. Together, these organs form a system that supports reproduction and hormonal balance.
# 43. It’s important to remember that every woman’s anatomy can look slightly different.
# 44. All variations are normal as long as the organs are healthy.
# 45. Maintaining reproductive health involves hygiene, medical checkups, and understanding your body.
# 46. The female reproductive system also plays roles beyond reproduction, such as hormone regulation.
# 47. It influences emotions, skin health, and bone strength.
# 48. Learning about these organs helps us respect and care for our bodies better.
# 49. Always feel free to ask questions about reproductive health without embarrassment.
# 50. That’s all for today’s lesson on the human female sexual organs.

# ---

# Would you like me to add **labeled diagrams or drawings** to go with this transcript (for classroom or print use)?

# """

In [ ]:
def generate_text_from_video():
    return ""

def generate_text_from_pdf():
    return ""

In [6]:
user_explain_concept(transcript, "Can you explain the function of the ovaries in simple terms?")

NameError: name 'transcript' is not defined

In [ ]:

# summarize_function ={
#     "name": "summarize_text",
#     "description": "Summarizes the given text into concise bullet points.",
#     "parameters": {
#         "type": "object",
#         "properties": {
#             "content": {
#                 "type": "string",
#                 "description": "A concise summary of the provided text in bullet points."
#             }
#         },
#         "required": ["content"]
#     }
# }

# user_explanation_function = {
#     "name": "explain_user_concept",
#     "description": "Explains a concept based on the user's challenge.",
#     "parameters": {
#         "type": "object",
#         "properties": {
#             "user_input": {
#                 "type": "string",
#                 "description": "The user's input or challenge that needs explanation."
#             }
#         },
#         "required": ["user_input"]
#     }
# }

In [ ]:
import gradio as gr
import time

# --- Flashcard Data ---
flashcards = [
    {"front": " ", "back": " "},
]

# --- Custom CSS for the flip animation ---
# This CSS is injected into the Gradio app via the `css` argument in gr.Blocks
custom_css = """
.flashcard-container {
    width: 300px;
    height: 200px;
    perspective: 1000px; /* Perspective for 3D effect */
    margin: auto;
}

.flashcard {
    position: relative;
    width: 100%;
    height: 100%;
    text-align: center;
    transition: transform 0.6s; /* Animation duration */
    transform-style: preserve-3d;
    box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2);
    border-radius: 10px;
}

.flashcard.flipped {
    transform: rotateY(180deg);
}

.flashcard-front, .flashcard-back {
    position: absolute;
    width: 100%;
    height: 100%;
    backface-visibility: hidden; /* Hide the back when face is shown */
    display: flex;
    justify-content: center;
    align-items: center;
    font-size: 1.5em;
    padding: 20px;
    box-sizing: border-box;
    border-radius: 10px;
}

.flashcard-front {
    background-color: #f1f1f1;
    color: black;
}

.flashcard-back {
    background-color: #4CAF50;
    color: white;
    transform: rotateY(180deg); /* Initial state for the back side */
}
"""

# --- Python Logic ---
# Global variables to manage the state
current_card_index = 0
is_flipped = False

def flip_card():
    global is_flipped
    is_flipped = not is_flipped
    # Return a new HTML string with the 'flipped' class added or removed
    card = flashcards[current_card_index]
    front_text = card["front"]
    back_text = card["back"]
    flipped_class = "flipped" if is_flipped else ""

    return f"""
    <div class="flashcard-container">
        <div class="flashcard {flipped_class}">
            <div class="flashcard-front">{front_text}</div>
            <div class="flashcard-back">{back_text}</div>
        </div>
    </div>
    """

def next_card():
    global current_card_index, is_flipped
    current_card_index = (current_card_index + 1) % len(flashcards)
    is_flipped = False # Reset to front for the new card
    return flip_card() # Generate HTML for the new card

# --- Gradio Interface ---
with gr.Blocks(css=custom_css, title="Animated Flashcards") as flashcard:
    gr.Markdown("## Animated Flashcards with Gradio")
    
    # Use an HTML component to render the custom flashcard structure
    card_display = ""
    if len(flashcards) > 0:
       card_display = gr.HTML(value=flip_card()) # Initial card

    with gr.Row():
        flip_btn = gr.Button("Flip Card")
        next_btn = gr.Button("Next Card")

    # Event listeners
    flip_btn.click(
        fn=flip_card,
        outputs=[card_display]
    )
    
    next_btn.click(
        fn=next_card,
        outputs=[card_display]
    )

if __name__ == "__main__":
   if len(flashcards) > 0: 
      flashcard.launch()


In [7]:
import gradio as gr

# Define function that returns a tuple (3 outputs)
def process_input(user_text):
    return (
        user_explain_concept(pdf_text, user_text),
        list_key_points(pdf_text) ,
        f"Output 3: Reversed text → {user_text[::-1]}"
    )
    
def view_pdf (pdf_url):
     return pdf_url
 
def render_local_pdf():
    if not os.path.exists(media_dic.get("pdf", "")):
        return f"<p style='color:red;'>⚠️ File not found: {media_dic.get('pdf', '')}</p>"

    # Create iframe to display PDF
    return f"<iframe src='./{media_dic.get('pdf', '')}' width='100%' height='600px'></iframe>"

# Build layout
with gr.Blocks() as demo:
    gr.Markdown("## Gradio 3x2 Custom Layout Example")

    # --- Row 1 ---
    with gr.Row():
        with gr.Column():
            gr.Markdown("### 📘 Topic: Learning with Gradio")
            show_button = gr.Button("Show PDF")
            pdf_display = gr.HTML()
            
            show_button.click(render_local_pdf, outputs=pdf_display)
        with gr.Column():
            out1 = gr.Textbox(label="A Quick Explanation", lines=10)

    # --- Row 2 ---
    with gr.Row():
        with gr.Column():
            # Embed a YouTube video using iframe
            gr.HTML("""
                <iframe width="500" height="300" 
                src="https://www.youtube.com/embed/3jZ5vnv-LZc"
                title="Gradio Tutorial" 
                frameborder="0" allowfullscreen></iframe>
            """)
        with gr.Column():
            out2 = gr.Textbox(label="Key Points", lines=10)
          
            # out2 = gr.Textbox(label="Output 2")

    # --- Row 3 ---
    with gr.Row():
        with gr.Column():
            text_input = gr.TextArea(label="Enter your text here")
            submit_btn = gr.Button("Submit")
        with gr.Column():
            out3 = gr.Textbox(label="Output 3")

    # --- Link function ---
    submit_btn.click(fn=process_input, inputs=text_input, outputs=[out1, out2, out3] )

# Launch app
demo.launch()



* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
